In [1]:
import pandas as pd

In [5]:
kody_df = pd.read_csv("./data/ciselniky/CZ301VYK.csv", sep=";", encoding="windows-1250")

kody_doplnek_df = pd.read_csv("./data/ciselniky/CZ302VYK.csv", sep=";", encoding="windows-1250")
kody_df = pd.merge(kody_df, kody_doplnek_df, on="KOD", how="left")

kody_df.head()



,KOD,PLATIOD,PLATIDO,NAZ,ODB,OME,OMO,ZUM,TVY,CTN,...,PJP,BOD,KAT,UMA,UBO,AGREG,_1_20240706085845_x,TYP,TEXT,_1_20240706085845_y
0,41,19970701,30001231,JEDEN DEN POBYTU PACIENTA VE STACIONÁŘI I.,000,A,NaN,NaN,480.0,49.0,...,NaN,175.0,NaN,NaN,NaN,NaN,NaN,1.0,"Vykazuje se, pokud délka pobytu dosáhne alespo...",NaN
1,42,20090101,30001231,JEDEN DEN POBYTU PACIENTA VE STACIONÁŘI II.,000,A,NaN,NaN,180.0,25.0,...,NaN,87.0,NaN,NaN,NaN,NaN,NaN,1.0,"Vykazuje se, za 1 den pobytu pacienta ve staci...",NaN
2,43,20230101,30001231,OŠETŘOVACÍ DEN V OTEVŘENÉM DENNÍM STACIONÁŘI S...,000,A,NaN,NaN,NaN,170.0,...,NaN,1250.0,NaN,NaN,NaN,NaN,NaN,1.0,OŠETŘOVACÍ DEN V OTEVŘENÉM DENNÍM STACIONÁŘI S...,NaN
3,43,20230101,30001231,OŠETŘOVACÍ DEN V OTEVŘENÉM DENNÍM STACIONÁŘI S...,000,A,NaN,NaN,NaN,170.0,...,NaN,1250.0,NaN,NaN,NaN,NaN,NaN,1.0,Péče je poskytována na základě indikace psychi...,NaN
4,130,19970701,30001231,DLOUHODOBÁ MIMOTĚLNÍ RESPIRAČNÍ PODPORA NOVORO...,304,S,NaN,Z,480.0,1920.0,...,NaN,21382.0,NaN,NaN,NaN,NaN,NaN,1.0,Dlouhodobá mimotělní respirační podpora ECLS (...,NaN


In [6]:
used_kody = pd.read_csv("./data/vykazy/vyk_23_vykony_new.csv", sep=";", encoding="windows-1250")



In [8]:
kody_df_used_only = kody_df[kody_df["KOD"].isin(used_kody["KOD"])][["KOD", "NAZ", "TEXT", "ODB"]].reset_index(drop=True)
kody_df_used_only.head()

kody_df_used_only.columns = ["code", "name", "description", "odbornost"]
with open("./data/ciselniky/vykon.jsonl", "w", encoding="utf-8") as file:
    kody_df_used_only.to_json(file, force_ascii=False, orient="records", lines=True)
    # kody_df_used_only.to_json("./data/ciselniky/vykon.jsonl", orient="records", lines=True)

In [14]:
len(kody_df_used_only)

102

In [27]:
leky_df = pd.read_csv("./data/ciselniky_material/CZ211LEK.csv", sep=";", encoding="windows-1250")
leky_df["file"] = "Hromadně vyráběné léčivé přípravky"
leky_df["file_code"] = "01"
# leky_df.head()

ivl_df = pd.read_csv("./data/ciselniky_material/CZ212IVL.csv", sep=";", encoding="windows-1250")
ivl_df["file"] = "Individuálně připravované léčivé přípravky a výrobky transfuzních stanic a radiofarmaka"
ivl_df["file_code"] = "02"
# ivl_df.head()

szm_df = pd.read_csv("./data/ciselniky_material/CZ213SZM.csv", sep=";", encoding="windows-1250")
szm_df["file"] = "Zdravotnické prostředky"
szm_df["file_code"] = "03"
# leky_df.columns = ["code", "name", "description", "odbornost"]
# leky_df.to_json("./data/ciselniky/lek.jsonl", orient="records", lines=True)
# len(leky_df)

materialy_df = pd.concat([leky_df, ivl_df, szm_df], axis=0, ignore_index=True)
materialy_df = materialy_df[["KOD", "NAZ", "DOP", "file", "file_code"]].reset_index(drop=True)
materialy_df.columns = ["code", "name", "description", "file", "file_code"]
materialy_df.head()
# materialy_df.columns = ["code", "name", "description", "odbornost"]
materialy_df.to_json("./data/ciselniky/materialy.jsonl", orient="records", lines=True)
# len(materialy_df)



/var/folders/sf/yy_tgz8n46x94kfzmknffnjm0000gn/T/ipykernel_15834/2667646204.py:1: DtypeWarning: Columns (0,10,11,34) have mixed types. Specify dtype option on import or set low_memory=False.
  leky_df = pd.read_csv("./data/ciselniky_material/CZ211LEK.csv", sep=";", encoding="windows-1250")
/var/folders/sf/yy_tgz8n46x94kfzmknffnjm0000gn/T/ipykernel_15834/2667646204.py:11: DtypeWarning: Columns (7,11,29,35,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  szm_df = pd.read_csv("./data/ciselniky_material/CZ213SZM.csv", sep=";", encoding="windows-1250")


In [9]:
used_diag = pd.read_csv("./data/vykazy/vyk_23_vykony_new.csv", sep=";", encoding="windows-1250")
diag_full = pd.read_csv("./data/ciselniky/CZ110DGA.csv", sep=";", encoding="windows-1250")

In [10]:
used_diag = pd.merge(used_diag, diag_full, left_on="DG", right_on="KOD", how="left")
used_diag = used_diag[["DG", "NAZ"]].drop_duplicates()

with open("./data/ciselniky/mkn.jsonl", "w", encoding="utf-8") as file:
    used_diag.to_json(file, force_ascii=False, orient="records", lines=True)
# used_diag.groupby("ODBORNOST")["DG"].value_counts()
# used_diag["DG"].value_counts()
